In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import networkx as nx

import graph_delineator.delineate as gd

In [ ]:
# Define MERIT data directories
merit_dirs = {
    'basins': Path('/nas/cee-ice/data/MERIT'),
    'flow_dir': Path('/nas/cee-ice/data/MERIT-Hydro/processed/dir/'),
    'flow_acc': Path('/nas/cee-ice/data/MERIT-Hydro/processed/upg/')
}
gauges_csv = Path('/nas/cee-water/cjgleason/ted/graph_delineator/data/clamped_manual_matchups.csv')
output_dir = Path('/nas/cee-water/cjgleason/ted/graph_delineator/data/clamped_manual_matchups_500')

In [ ]:
gd.delineate_basins(gauges_csv, merit_dirs, 500, save_plots=True, output_dir=output_dir)

In [ ]:
debug

In [ ]:
from tqdm import tqdm

basin_file_dir = output_dir / 'subbasins'
subs_list = []
for f in tqdm(basin_file_dir.glob("*.parquet")):
    outlet_id = f.stem.split('_subbasins')[0]
    sub_gdf = gpd.read_parquet(f).rename(columns={'id':'site_id'})
    sub_gdf['outlet_id'] = outlet_id
    subs_list.append(sub_gdf)
subbasins = pd.concat(subs_list).set_index('site_id')

subbasins

In [ ]:

gdf = gpd.read_parquet('/nas/cee-water/cjgleason/ted/swot-ml/data/multigraph_manual/metadata/subbasins_500.parquet')

In [ ]:
subbasins = gdf
subbasins

In [ ]:
target = 'USGS-07374000'
subbasins = pd.concat([
    subbasins[subbasins["outlet_id"] == target], 
    subbasins[subbasins["outlet_id"] != target]
])
subbasins

In [ ]:
from data import BasinDataLake

store = BasinDataLake(args.save_dir)
status = store.get_processing_status(source='era5')
processed_basins = [] if status.empty else status.index.get_level_values('basin').unique()
to_process = subbasins[subbasins['outlet_id'].isin(processed_basins)]

to_process

In [ ]:

subbasins.to_parquet("/nas/cee-water/cjgleason/ted/swot-ml/data/multigraph_manual/metadata/subbasins_500.parquet")

In [ ]:
"UKEA-694039" # single gauge with weird location

In [ ]:
all_gauges = gd.load_gauges(gauges_csv)
megabasins_outlets = gd.get_pfaf1_outlet_dict(all_gauges)

b = False
for megabasin_id, outlet_dict in megabasins_outlets.items():
    for outlet_id, outlet_gauge_ids in outlet_dict.items():   

        if outlet_id != 'EAUF-E5300213':
            continue
            
        print(outlet_id)
        pfaf1_catchments, pfaf1_rivers = gd.load_basin_data(megabasin_id, merit_dirs)
        
        gauges = all_gauges[all_gauges["id"].isin(outlet_gauge_ids)]
        gauges = gd.set_gauge_area_range(gauges, pfaf1_rivers)

        final_comid = gauges[gauges['id'] == outlet_id]['COMID'].item()
        comids = gd.get_network_comids(final_comid,  pfaf1_rivers)

        catchments = pfaf1_catchments.loc[pfaf1_catchments.index.isin(comids)]
        rivers = pfaf1_rivers.loc[pfaf1_rivers.index.isin(comids)]
        
        G = gd.build_graph_with_geometries(catchments, rivers)

        b = True
        break
    
    if b:
        break

In [ ]:
catchments.plot()

In [ ]:
gauge_info = {}
node_replacements = {}

gauges['node_id'] = gauges['COMID'].astype(str)
gauges = gauges.sort_values(['node_id', 'position'], ascending=False)
gauges

In [ ]:
from graph_delineator.split_catchment import split_catchment_raster

In [ ]:
merit_dirs

In [ ]:
pfaf2_id = 23
MERIT_RES = 0.000833333  # 3 arc second resolution in degrees
TINY_AREA_THRESHOLD_KM2 = 0.5 

for _, gauge in gauges.iterrows():
    gauge_id = str(gauge["id"])

    # Follow the replacement chain to find current node
    original_node_id = gauge["node_id"]
    current_node_id = original_node_id
    while current_node_id in node_replacements:
        current_node_id = node_replacements[current_node_id]

    # Get the current polygon (may have been split already)
    current_polygon = G.nodes[current_node_id]["polygon"]

    gauge_polygon, is_leaf = split_catchment_raster(
        gauge=gauge.to_dict(),
        basin=pfaf2_id,
        catchment_poly=current_polygon,
        flow_dir_path=merit_dirs['flow_dir'],
        flow_acc_path=merit_dirs['flow_acc'],
    )

    is_leaf = False

    
    if gauge_polygon is None:
        # Failed, reason will be logged by fn
        continue

    if gauge_polygon == current_polygon:
        # No split needed. Convert the node to a gauge
        gd.convert_node_to_gauge(G, gauge_id, gauge["lat"], gauge["lng"], current_node_id)
        # Track that this node was replaced
        node_replacements[current_node_id] = gauge_id
        gauge_info[gauge_id] = {
            "original_node": original_node_id, 
            "current_node": current_node_id, 
            "method": "replace",
        }
        continue

    # Calculate remainder
    remainder_polygon = current_polygon.difference(gauge_polygon)
    remainder_polygon = remainder_polygon.buffer(-MERIT_RES / 2).buffer(MERIT_RES / 2)

    if is_leaf:
        gd.add_leaf_gauge_to_graph(
            G,
            gauge_id,
            gauge['lat'],
            gauge['lng'],
            gauge_polygon,
            remainder_polygon,
            current_node_id
        )
    else:
        gd.insert_gauge_into_graph(
            G,
            gauge_id,
            gauge["lat"],
            gauge["lng"],
            gauge_polygon,
            remainder_polygon,
            current_node_id,
        )
        gauge_info[gauge_id] = {"original_node": original_node_id, "current_node": current_node_id, "method": "split"}


    break


In [ ]:
subs, g = gd.DelineationResult(G, '').to_geodataframes()

In [ ]:
subs

In [ ]:
from graph_delineator.split_catchment import split_catchment_raster

In [ ]:
comids

In [ ]:
min(3, 1, 2)

In [ ]:
G_backup = G.copy()

In [ ]:
G = G_backup.copy()
gauges_gdf = gauges
flow_dir_path = merit_dirs['flow_dir']
basin_id = megabasin_id

gauge_info = {}
# Track node replacements: old_node_id -> new_gauge_id
node_replacements = {}

# Find which catchment each gauge falls in (INITIAL spatial join only)
catchment_polys = []
node_ids = []
for node_id, data in G.nodes(data=True):
    if data.get("polygon"):
        catchment_polys.append(data["polygon"])
        node_ids.append(node_id)

catchments_gdf = gpd.GeoDataFrame(
    {"node_id": node_ids}, geometry=catchment_polys, crs=gauges_gdf.crs
)

ax = catchments_gdf.plot('node_id', edgecolor='black', linewidth=0.8)
gauges.plot(color='black', ax=ax)

In [ ]:
gauges_with_nodes = gpd.sjoin(
    gauges_gdf, catchments_gdf, how="left", predicate="within"
)
# Process from upstream (0.0) to downstream (1.0)
gauges_with_nodes = gauges_with_nodes.sort_values(['node_id', 'position'])
gauge_iter = gauges_with_nodes.iterrows()

In [ ]:
_, gauge = next(gauge_iter)
gauge_id = gauge["id"]
river_position = gauge.get("position")

# Follow the replacement chain to find current node
original_node_id = gauge["node_id"]
current_node_id = original_node_id
while current_node_id in node_replacements:
    current_node_id = node_replacements[current_node_id]

# Get the current polygon (may have been split already)
current_polygon = G.nodes[current_node_id]["polygon"]
current_polygon

In [ ]:
catchment_poly = current_polygon
bounds = catchment_poly.bounds
bounds_list = [float(i) for i in bounds]

bounds_list

In [ ]:
from numpy import ceil, floor
MERIT_RES = 0.000833333  # 3 arc second resolution in degrees
halfpix = MERIT_RES / 2  # Half pixel width

# Adjust bounds to pixel centers
bounds_list[0] = floor(bounds_list[0] * 1200) / 1200 - halfpix
bounds_list[1] = floor(bounds_list[1] * 1200) / 1200 - halfpix
bounds_list[2] = ceil(bounds_list[2] * 1200) / 1200 + halfpix
bounds_list[3] = ceil(bounds_list[3] * 1200) / 1200 + halfpix
bounding_box = tuple(bounds_list)
bounds_list

In [ ]:
megabasin_id

In [ ]:
import numpy as np
from pysheds.grid import Grid

pfaf2_id = int(final_comid//1E6)

flow_file = str(flow_dir_path / f"{pfaf2_id}.tif")

grid = Grid.from_raster(flow_file, window=bounding_box, nodata=0)
fdir = grid.read_raster(flow_file, window=bounding_box, nodata=0)

facc_file = str(merit_dirs['flow_acc'] / f"{pfaf2_id}.tif")
facc = grid.read_raster(facc_file, window=bounding_box, nodata=0)


mymask = grid.rasterize([catchment_poly])

fdir[mymask == 0] = 0
facc[mymask == 0] = 0
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)  # ESRI flow direction standard

plt.imshow(fdir)
plt.show()
plt.imshow(np.log10(facc+0.5))
plt.show()

In [ ]:
fdir.min()

In [ ]:


get_pixel_area_m2(gauge, grid)

In [ ]:
min_pixels = min_area * 1E6 / pixel_area_m2
min_pixels

x_snap, y_snap = grid.snap_to_mask(facc > min_pixels, (gauge['lng'], gauge['lat']))

x_snap

In [ ]:
gauge.to_dict()

In [ ]:
catch = grid.catchment(
    fdir=fdir,
    x=x_snap, 
    y=y_snap,
    dirmap=dirmap,
    xytype="coordinate",
    recursionlimit=15000,
)

plt.imshow(catch)

In [ ]:
grid.clip_to(catch)
clipped_catch = grid.view(catch, dtype=np.uint8)

In [ ]:
plt.imshow(facc)

In [ ]:
plt.imshow(facc > 100)

In [ ]:
facc * catch

In [ ]:
np.log10((facc * catch)+0.5)

In [ ]:
plt.imshow(np.log10((facc * catch)+0.5))

In [ ]:
import scipy.ndimage as ndi

# 2. Erode the mask. This "shrinks" it by one pixel.
#    Pixels that were on the edge are set to False.
eroded_mask = ndi.binary_erosion(catch)

# 3. Find the edge by selecting pixels that are in the
#    original mask but *not* in the eroded mask.
catch_edge = catch & ~eroded_mask

# 4. Find pixels that are on the catchment edge AND have acc > 1
outlets_on_edge = (facc > 100) | catch_edge*2

# 5. Count them
num_outlets_on_edge = np.sum(outlets_on_edge)
print(num_outlets_on_edge)

plt.imshow(outlets_on_edge)

In [ ]:
plt.imshow(catch_edge)

In [ ]:
acc

In [ ]:
plt.imshow(outlets_on_edge)

In [ ]:
type(catch)

In [ ]:
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

# 1. Create the edge mask (as you did)
eroded_mask = ndi.binary_erosion(catch)
catch_edge = catch & ~eroded_mask

# 2. STRICT FILTER: Intersection of Edge AND High Accumulation
# Use '&' not '|'. Adjust threshold (100) based on your resolution.
significant_edge_flow = catch_edge & (facc >= (min_pixels*0.95))

# 3. Label the clusters (Connected Components)
# structure=np.ones((3,3)) allows diagonal connections (8-connectivity)
labeled_array, num_features = ndi.label(significant_edge_flow, structure=np.ones((3,3)))

print(f"Number of flow locations found: {num_features}")

# Visual check
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(catch)
plt.title("Catchment")
plt.subplot(122)
plt.imshow(labeled_array, cmap='nipy_spectral', interpolation='nearest')
plt.title(f"Identified Clusters: {num_features}")
plt.show()

In [ ]:
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

# Define ESRI directional offsets: (dy, dx)
# 64:N, 128:NE, 1:E, 2:SE, 4:S, 8:SW, 16:W, 32:NW
esri_offsets = {
    64:  (-1, 0),  128: (-1, 1),
    1:   (0, 1),   2:   (1, 1),
    4:   (1, 0),   8:   (1, -1),
    16:  (0, -1),  32:  (-1, -1)
}

# 1. Erode to find edges
eroded_mask = ndi.binary_erosion(catch)
catch_edge = catch & ~eroded_mask

# 2. Filter: Edge pixels AND High Accumulation (Fixing the bitwise operator)
# Adjust '100' based on your specific resolution/stream threshold requirements
significant_edge_flow = catch_edge & (facc > 100)

# 3. Label clusters
labeled_array, num_features = ndi.label(significant_edge_flow, structure=np.ones((3,3)))

print(f"Found {num_features} flow connections on the edge.")

inflow_count = 0
outflow_count = 0

# 4. Iterate through each cluster to classify
for i in range(1, num_features + 1):
    # Get coordinates of all pixels in this cluster
    y_locs, x_locs = np.where(labeled_array == i)
    
    # Find the pixel with the MAXIMUM flow accumulation in this cluster
    # This is the most reliable point to check (center of the stream)
    cluster_faccs = facc[y_locs, x_locs]
    max_idx = np.argmax(cluster_faccs)
    
    py, px = y_locs[max_idx], x_locs[max_idx]
    direction = fdir[py, px]
    
    # Get the offset for this direction
    dy, dx = esri_offsets.get(direction, (0, 0))
    
    if dy == 0 and dx == 0:
        print(f"Cluster {i}: Sink or Invalid Direction (Val: {direction})")
        continue

    # Calculate target coordinates
    ny, nx = py + dy, px + dx
    
    # Check bounds and mask status
    h, w = catch.shape
    if 0 <= ny < h and 0 <= nx < w:
        target_in_catchment = catch[ny, nx]
    else:
        # If it flows off the grid, it's definitely leaving the catchment
        target_in_catchment = False

    # CLASSIFICATION LOGIC
    if target_in_catchment:
        # If the edge pixel flows INTO the mask, it is an INFLOW
        # (Water arrived here from outside and is continuing in)
        print(f"Cluster {i}: INFLOW (Max Acc: {cluster_faccs[max_idx]})")
        inflow_count += 1
    else:
        # If the edge pixel flows OUT of the mask, it is an OUTFLOW
        print(f"Cluster {i}: OUTFLOW (Max Acc: {cluster_faccs[max_idx]})")
        outflow_count += 1

print(f"Summary: {inflow_count} Inflows, {outflow_count} Outflows")

# Visual check
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(catch)
plt.title("Catchment")
plt.subplot(122)
plt.imshow(labeled_array, cmap='nipy_spectral', interpolation='nearest')
plt.title(f"Identified Clusters: {num_features}")
plt.show()

In [ ]:
num_features

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Adjust your figure size
plt.figure(figsize=(12, 6))

# Iterate through the clusters found in the previous step
for i in range(1, num_features + 1):
    # 1. Find bounds of this cluster to zoom in
    y_c, x_c = np.where(labeled_array == i)
    
    # Add padding to view context
    pad = 5
    y_min, y_max = np.min(y_c) - pad, np.max(y_c) + pad
    x_min, x_max = np.min(x_c) - pad, np.max(x_c) + pad
    
    # 2. Identify the Critical Pixel (Max Flow Acc)
    cluster_faccs = facc[y_c, x_c]
    max_idx = np.argmax(cluster_faccs)
    py, px = y_c[max_idx], x_c[max_idx]
    
    # 3. Get Flow Vector
    direction = fdir[py, px]
    dy, dx = esri_offsets.get(direction, (0,0))
    
    # Setup Subplot
    ax = plt.subplot(1, num_features, i)
    
    # Plot the Catchment Mask (Yellow=In, Purple=Out)
    subset_mask = catch[y_min:y_max, x_min:x_max]
    ax.imshow(subset_mask, cmap='viridis', alpha=0.6, origin='upper', extent=[x_min, x_max, y_max, y_min])
    
    # Plot the Cluster Pixels (Red)
    ax.scatter(x_c, y_c, c='red', s=10, label='Cluster')
    
    # Plot the Flow Arrow from the Max Facc Pixel
    # Note: Quiver (X, Y, U, V) -> U corresponds to dx (cols), V to -dy (rows, because image y is down)
    ax.quiver(px + 0.5, py + 0.5, dx, -dy, color='white', scale=1, scale_units='xy', angles='xy')
    
    # Highlight the target pixel
    target_y, target_x = py + dy, px + dx
    is_in = catch[target_y, target_x]
    status = "IN (Inflow)" if is_in else "OUT (Outflow)"
    f
    ax.set_title(f"Cluster {i}\nTarget is {status}")
    ax.invert_yaxis() # Align with image coordinates

plt.tight_layout()
plt.show()

In [ ]:
facc

In [ ]:
import numpy as np

print(np.unique(fdir))

In [ ]:
from shapely import wkb
from shapely.geometry import Polygon, MultiPolygon


mymask = grid.rasterize([catchment_poly])

fdir = grid.read_raster(flow_file, window=bounding_box, nodata=0)
m, n = grid.shape
for i in range(m):
    for j in range(n):
        if int(mymask[i, j]) == 0:
            fdir[i, j] = 0

plt.imshow(fdir)

In [ ]:
catch = grid.catchment(
    fdir=fdir,
    x=gauge['lng'],
    y=gauge['lat'],
    dirmap=dirmap,
    xytype="coordinate",
    recursionlimit=15000,
)
plt.imshow(catch)

In [ ]:
catch

In [ ]:
gauge_polygon = split_catchment_raster(
    gauge_id=gauge_id,
    basin=basin_id,
    lat=gauge["lat"],
    lng=gauge["lng"],
    river_position=river_position,
    catchment_poly=current_polygon,
    flow_dir_path=flow_dir_path,
)

In [ ]:
MERIT_RES = 0.000833333  # 3 arc second resolution in degrees

remainder_polygon = current_polygon.difference(gauge_polygon)
remainder_polygon = remainder_polygon.buffer(-MERIT_RES / 2).buffer(MERIT_RES / 2)

remainder_polygon

In [ ]:
MERIT_RES * MERIT_RES

In [ ]:
gauge_polygon.area

In [ ]:
comids

In [ ]:
subbasins, gauges = results['USGS-01059000'].to_geodataframes()

In [ ]:
ax = subbasins[subbasins['is_gauge']].plot('id', alpha=0.8, cmap='inferno')
subbasins.plot(color='grey', ax=ax, zorder=0)
gauges.plot('id', ax=ax, cmap='inferno')#, style_kwds={'markeredgecolor':'black'})

In [ ]:
output_dir = Path("/nas/cee-water/cjgleason/ted/graph_delineator/outputs/test")
gd.save_results(results, output_dir, 'test')